In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm
from tqdm import tqdm_notebook
import warnings
from sklearn.preprocessing import LabelEncoder
import gc
import os
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from bayes_opt import BayesianOptimization

plt.style.use('seaborn')
sns.set(font_scale=1)
pd.set_option('display.max_columns', 500)

In [2]:
train = pd.read_csv('train.csv')
label = pd.read_csv('train_label.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('submission.csv')

# 数据探索

In [3]:
df_source = train.merge(label,on='ID',how='left')

# 特征工程

In [4]:
test['label'] = -1
train = train.merge(label,on='ID',how='left')
data = pd.concat([train, test])

In [5]:
def get_time_fe(df):
    df['day'] = df.date.apply(lambda x:int(x[8:10]))

    df['hour'] = df.date.apply(lambda x:int(x[11:13]))

    return df

In [6]:
#对时间分箱
def getSeg(x):
    if x >=0 and x<= 6:
        return 1
    elif x>=7 and x<=12:
        return 2
    elif x>=13 and x<=18:
        return 3
    elif x>=19 and x<=23:
        return 4

In [7]:
# hourDF = train_df.groupby(['hour', 'label'])['hour'].count().unstack('label').fillna(0)
# hourDF[[0,1]].plot(kind='bar', stacked=True);

In [8]:
#count统计特征
cross_feature = []
def get_cross_fe(df):
    first_feature = ['B1', 'B2', 'B3']
    second_feature = ['C1','C2','C3','D1','D2']
    for feat_1 in first_feature:
        for feat_2 in second_feature:
            col_name = "cross_" + feat_1 + "_and_" + feat_2
            cross_feature.append(col_name)
            df[col_name] = df[feat_1].astype(str).values + '_' + df[feat_2].astype(str).values
    return df

In [9]:
data = get_time_fe(data)
# data['hour_seg'] = data['hour'].apply(lambda x: getSeg(x))
data = get_cross_fe(data)

In [10]:
#labelencoder()
cate_feature = ['A1','A2','A3','B1','B2','B3','C1','C2','C3','E2','E3','E5','E7','E9','E10','E13','E16','E17','E19','E21','E22']
cross_feature = cross_feature[:15]
cate_features = cate_feature+cross_feature
for item in cate_features:
    data[item] = LabelEncoder().fit_transform(data[item])

In [11]:
def feature_count(data, features=[]):
    new_feature = 'count'
    for i in features:
        new_feature += '_' + i
    try:
        del data[new_feature]
    except:
        pass
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    data = data.merge(temp, 'left', on=features)
    return data

for i in cross_feature:
    n = data[i].nunique()
    if n > 5:
        data = feature_count(data, [i])
    else:
        print(i, ':', n)

In [12]:
#ratio：类别偏好的ratio比例特征
label_feature =[ 'A2', 'A3']
data_temp = data[label_feature]
df_feature = pd.DataFrame()
data_temp['cnt'] = 1
print('Begin ratio clcik...')
col_type = label_feature.copy()
n = len(col_type)
for i in range(n):
    col_name = "ratio_click_of_" + col_type[i]
    df_feature[col_name] = (
                    data_temp[col_type[i]].map(data_temp[col_type[i]].value_counts()) / len(data) * 100).astype(int)            
data = pd.concat([data, df_feature], axis=1)
print('The end')

Begin ratio clcik...
The end


In [13]:
len(data.D2.unique())

3

In [14]:
train_df = data[data['label'] != -1]
test_df = data[data['label'] == -1]

In [15]:
# sns.stripplot(train_df["label"],train_df["A3"],jitter=True,order=None)
# plt.title("click Vs creative_height");

# 删除不需要的字段

In [16]:
## get train feature
del_feature = ['ID','day','date','label','D2']+cross_feature
features = [i for i in train_df.columns if i not in del_feature]

In [17]:
train_x = train_df[features]
train_y = train_df['label'].values
test = test_df[features]

In [ ]:
params= {
   'num_leaves':97,
    'objective':'binary',
    'learning_rate':0.02,
   'max_bin':425,
   'min_data_in_leaf':10,
    'feature_fraction':0.5,
    'bagging_fraction':0.9,
    
    'subsample':0.8,
    'subsample_freq':1,
   'bagging_freq':1,
   'bagging_seed':0,
    'min_split_gain':0.000135,
    
    'min_child_weight':5,
    'min_child_samples':30,
    
    'metric': {'binary_logloss', 'auc'},
   'reg_alpha':3,
   'reg_lambda':5
}

In [20]:
data = lgb.Dataset(train_x,train_y)

def lgb_cv(num_leaves,learning_rate,max_bin, min_data_in_leaf,feature_fraction,bagging_fraction,subsample,max_depth,min_split_gain,min_child_weight,min_child_samples,reg_alpha,reg_lambda,num_iterations=1000):
        
        params = {'objective':'binary','num_iterations': num_iterations, 'early_stopping_round':50, 'metric':{'binary_logloss', 'auc'}}
        params["num_leaves"] = int(round(num_leaves))
        params["learning_rate"] = learning_rate
        params["max_bin"] = int(round(max_bin))
        params["min_data_in_leaf"] = int(round(min_data_in_leaf))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['subsample'] = max(min(subsample, 1), 0)
        params['max_depth'] = int(round(max_depth))
        
        params['min_split_gain'] = min_split_gain      
        
        params['min_child_weight'] = int(round(min_child_weight))
        params['min_child_samples'] = int(round(min_child_samples))
        
        params['reg_alpha'] = int(round(reg_alpha))
        params['reg_lambda'] = int(round(reg_lambda))

        
        cv_result = lgb.cv(params, data, nfold=5, seed=2, stratified=True, verbose_eval =50)
        return -(min(cv_result['auc-mean']))

lgb_bo = BayesianOptimization(
        lgb_cv,
        {'num_leaves': (16, 128),
         'learning_rate': (0, 0.5),
         'max_bin': (300, 500),
         'min_data_in_leaf': (1, 100),    
        'feature_fraction': (0, 1),
        'bagging_fraction': (0.5, 1),
        'subsample': (0.5, 1),
         'max_depth': (0, 20),
         'min_split_gain': (0, 5),
         'min_child_weight': (0,20),
         'min_child_samples':(0,100),
         
          'reg_alpha':(0,20),
         'reg_lambda':(0,20)})

lgb_bo.maximize(init_points=21,n_iter=90) #init_points表示初始点，n_iter代表迭代次数（即采样数）
print (lgb_bo.max)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_ch... | min_ch... | min_da... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[50]	cv_agg's binary_logloss: 0.41169 + 0.00184543	cv_agg's auc: 0.721175 + 0.00481312
|  1        | -0.6883   |  0.9284   |  0.6318   |  0.1294   |  447.7    |  17.34    |  54.06    |  13.54    |  4.566    |  1.253    |  52.71    |  0.2516   |  5.756    |  0.7997   |
[50]	cv_agg's binary_logloss: 0.415513 + 0.000926282	cv_agg's auc: 0.71156 + 0.00330583
|  2        | -0.6359   |  0.6584   |  0.461    |  0.3938   |  497.7    |  3.902    |  0.1211   |  1.085    |  45.71    |  2.326    |  99.84    |  0.2288   |  17.9     |  0.7043   |
[50]	cv_agg's binary_logloss: 0.417649 + 0.000789244	cv_agg's auc: 0.707966 + 0.0036329
[100]	cv_

KeyboardInterrupt: 

# 训练参数的选取

In [ ]:
# params= {
#    'num_leaves':127,
#     'objective':'binary',
#     'learning_rate':0.035,
#    'max_bin':425,
#    'min_data_in_leaf':10,
#     'feature_fraction':0.5,
    
#    'bagging_freq':1,
#    'bagging_seed':0,
#     'bagging_fraction':0.9,
#     'metric': {'binary_logloss', 'auc'},

#    'reg_alpha':3,
#    'reg_lambda':5
# }